In [1]:
import torch
import transformers
import argparse
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from peft import (
        get_peft_model, 
        prepare_model_for_kbit_training, 
        LoraConfig
    )
from trl import SFTTrainer

In [2]:
model_name = "meta-llama/Llama-2-7b-hf"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                            load_in_8bit=True,
                                            device_map="auto"
                                            )
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# Load your custom JSON dataset
custom_data = load_dataset('json', data_files='selected_data.json')

# Access train, test, and validation splits if available
data_train = custom_data['train']

custom_data = load_dataset('json', data_files='data.json')
data_val = custom_data['train']

# Print the dataset details
print(data_train)
print(data_val)

# Access an example
#example = data_train[0]
#print(example)

def generate_prompt(title, abstract=None, eos_token="</s>"):
  instruction = "The abstract of the paper:\n"
  input = f"{title}\n"
  abstract = f"Abstract: {abstract + ' ' + eos_token if abstract else ''} "
  prompt = (" ").join([instruction, input, abstract])
  return prompt

print(generate_prompt(data_train[0]["title"], data_train[0]["abstract"]))

Dataset({
    features: ['publicationDate', 'title', 'abstract'],
    num_rows: 32654
})
Dataset({
    features: ['title', 'abstract', 'keyword'],
    num_rows: 278
})
The abstract of the paper:
 Magnetically tuned Ni_0.3Co_0.7Dy_xFe_2–xO_4 ferrites for high-density data storage applications
 Abstract: Dysprosium (Dy^3+)-substituted Ni–Co nanoparticles were synthesized by sol–gel technique. Structural and morphological analyses were accomplished by X-ray diffraction (XRD), scanning electron microscopy (SEM) and field emission transmission electron microscopy (FE-TEM). The crystallite size and lattice parameter followed a decreasing trend up on increase in Dy^3+ substitution for the concentration x  ≤ 0.15, which is due to the hindrance in crystallite growth and deposition of Dy^3+ on grain boundaries, respectively. The lattice strain was increased from 5.027 to 8.814 × $${10}^{-3}$$ 10 - 3 with enhancement in Dy^3+ content. The morphological studies showed uniform distribution of parti

In [7]:
input_prompt = generate_prompt(data_train[50]["title"])
input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")
with torch.cuda.amp.autocast():
  generation_output = model.generate(
      input_ids=input_tokens,
      max_new_tokens=1000,
      do_sample=True,
      top_k=10,
      top_p=0.9,
      temperature=0.3,
      repetition_penalty=1.15,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
    )
op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
print(op)

The abstract of the paper:
 Characteristic Research of Magnetic Controllable Voltage Regulator
 Abstract:   In this paper, we design a magnetic controllable voltage regulator. It is composed by two parts, one part is the control circuit and another part is the power circuit. The control circuit is made up of an LC oscillator, a comparator and some passive components. When the output voltage of the LC oscillator exceeds the reference voltage, the comparator will turn on the MOSFET to cut off the current flowing through it. And when the output voltage of the LC oscillator drops below the reference voltage, the comparator turns off the MOSFET to let the current flow through it again. The power circuit is made up of a transformer with three windings, which are the primary winding, secondary winding and tertiary winding respectively. The primary winding is connected in series with the load. The secondary winding is used for generating the high frequency signal. The tertiary winding is used 

In [9]:
lora_config = LoraConfig(
        r=256,
        lora_alpha=512,
        lora_dropout=0.05,
        target_modules=["q_proj","k_proj","v_proj","o_proj"],
        bias="none",
        task_type="CAUSAL_LM",
    )

# this should be set for finutning and batched inference
tokenizer.add_special_tokens({"pad_token": "<PAD>"})
model.resize_token_embeddings(len(tokenizer))

# Loading in 8 bit ..."
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [10]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): LlamaForCausalLM(
          (model): LlamaModel(
            (embed_tokens): Embedding(32001, 4096)
            (layers): ModuleList(
              (0-31): 32 x LlamaDecoderLayer(
                (self_attn): LlamaAttention(
                  (q_proj): lora.Linear8bitLt(
                    (base_layer): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=4096, out_features=256, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=256, out_features=4096, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
      

In [11]:
output_dir = "cp"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
per_device_eval_batch_size = 4
eval_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 200
logging_steps = 200
learning_rate = 1e-4
max_grad_norm = 0.2
#max_steps = 50
warmup_ratio = 0.03
evaluation_strategy="epoch"
lr_scheduler_type = "constant"

training_args = transformers.TrainingArguments(
            output_dir=output_dir,
            per_device_train_batch_size=per_device_train_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            optim=optim,
            evaluation_strategy=evaluation_strategy,
            save_steps=save_steps,
            learning_rate=learning_rate,
            logging_steps=logging_steps,
            max_grad_norm=max_grad_norm,
            #max_steps=max_steps,
            warmup_ratio=warmup_ratio,
            group_by_length=True,
            lr_scheduler_type=lr_scheduler_type,
            ddp_find_unused_parameters=False,
            eval_accumulation_steps=eval_accumulation_steps,
            per_device_eval_batch_size=per_device_eval_batch_size,
        )

In [10]:
def formatting_func(prompt):
  output = []

  for d, s in zip(prompt["title"], prompt["abstract"]):
    op = generate_prompt(d, s)
    output.append(op)

  return output


trainer = SFTTrainer(
    model=model,
    train_dataset=data_train,
    eval_dataset=data_val,
    peft_config=lora_config,
    formatting_func=formatting_func,
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_args
)

# We will also pre-process the model by upcasting the layer norms in float 32 for more stable training
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

trainer.train()
trainer.save_model(f"{output_dir}/final")

# Step Training Loss Validation Loss
# 10 1.848200 1.746341
# 20 1.688300 1.696681
# 30 1.654500 1.698127
# 40 1.579400 1.652010
# 50 1.492600 1.701877

Map:   0%|          | 0/278 [00:00<?, ? examples/s]

Map:   0%|          | 0/278 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
10,2.098400,1.891765
20,1.891100,1.811098
30,1.814300,1.744567
40,1.705700,1.667625
50,1.717800,1.575743


/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during q

In [7]:
from peft import PeftModel

peft_model_id = "cp/checkpoint-1000"
peft_model = PeftModel.from_pretrained(model, peft_model_id, torch_dtype=torch.float16, offload_folder="lora_results/lora_7/temp")

In [9]:
input_prompt = generate_prompt(data_train[50]["title"])
input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")
with torch.cuda.amp.autocast():
    generation_output = peft_model.generate(
        input_ids=input_tokens,
        max_new_tokens=100,
        do_sample=True,
        top_k=10,
        top_p=0.9,
        temperature=0.3,
        repetition_penalty=1.15,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
      )
op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
print(op)
# Summarize the following:
#  Pitt: Hey Teddy! Have you received my message?
# Teddy: No. An email?
# Pitt: No. On the FB messenger.
# Teddy: Let me check.
# Teddy: Yeah. Ta!
#  Summary:   Pitt sent a message to Teddy on Facebook Messenger, but he didn't receive it yet. 

The abstract of the paper:
 Characteristic Research of Magnetic Controllable Voltage Regulator
 Abstract:   In this paper, a magnetic controllable voltage regulator is designed. It consists of a transformer and two inductors. The control circuit includes an auxiliary winding on the primary side of the transformer and a current-controlled switch on the secondary side. When the load changes, the output voltage can be adjusted by changing the duty cycle of the switch. By using the PID controller to control the duty cycle of the switch, the output voltage can be controlled accurately and


In [10]:
# this should be set for finutning and batched inference
tokenizer.add_special_tokens({"pad_token": "<PAD>"})
model.resize_token_embeddings(len(tokenizer))

Embedding(32001, 4096)

In [ ]:
# Loading in 8 bit ..."
model = prepare_model_for_kbit_training(model)
#model = get_peft_model(model, lora_config)

output_dir = "cp"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
per_device_eval_batch_size = 4
eval_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 200
logging_steps = 200
learning_rate = 1e-4
max_grad_norm = 0.2
#max_steps = 50
warmup_ratio = 0.03
evaluation_strategy="epoch"
lr_scheduler_type = "constant"

training_args = transformers.TrainingArguments(
            output_dir=output_dir,
            per_device_train_batch_size=per_device_train_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            optim=optim,
            evaluation_strategy=evaluation_strategy,
            save_steps=save_steps,
            learning_rate=learning_rate,
            logging_steps=logging_steps,
            max_grad_norm=max_grad_norm,
            #max_steps=max_steps,
            warmup_ratio=warmup_ratio,
            group_by_length=True,
            lr_scheduler_type=lr_scheduler_type,
            ddp_find_unused_parameters=False,
            eval_accumulation_steps=eval_accumulation_steps,
            per_device_eval_batch_size=per_device_eval_batch_size,
        )

def formatting_func(prompt):
  output = []

  for d, s in zip(prompt["title"], prompt["abstract"]):
    op = generate_prompt(d, s)
    output.append(op)

  return output


trainer = SFTTrainer(
    model=peft_model,
    train_dataset=data_train,
    eval_dataset=data_val,
    #peft_config=lora_config,
    formatting_func=formatting_func,
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_args
)

# We will also pre-process the model by upcasting the layer norms in float 32 for more stable training
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

trainer.train()
trainer.save_model(f"{output_dir}/final")

# Step Training Loss Validation Loss
# 10 1.848200 1.746341
# 20 1.688300 1.696681
# 30 1.654500 1.698127
# 40 1.579400 1.652010
# 50 1.492600 1.701877